In [48]:
import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
from trajectory import trajectory

In [4]:
data_train = pd.read_csv('../data/train.csv')
data_test = pd.read_csv('../data/track1/test.csv')

In [5]:
train_data = data_train.copy(deep=True)
test_data = data_test.copy(deep=True)

In [63]:
time = []
for t in train_data['epoch']:
    time.append(datetime.strptime(t,"%Y-%m-%dT%H:%M:%S.%f"))
time = np.array(time)
time -= min(time)

In [67]:
time = [t.total_seconds() for t in time]

In [68]:
train_data['epoch'] = time

In [70]:
def generate_labels(n):
    labels = []
    for i in range(0,n):
        labels.append(i%24 + 1)
    return labels

In [71]:
cluster_labels = []
for sat_id in range(0,600):
    sat_data = data_train[data_train['sat_id'] == sat_id]
    cluster_labels += generate_labels(len(sat_data))

In [72]:
train_data['Cluster Label'] = cluster_labels

In [73]:
import visdom

In [74]:
vis = visdom.Visdom()

Setting up a new session...


In [112]:
def plot_axis(sat_id, axis, ellipse_index, span=1):
    data = train_data[train_data['sat_id'] == sat_id]
    label = data['Cluster Label']
    time = data['epoch']
    data = data.loc[:, ['x_sim', 'y_sim', 'z_sim']].values
    vis.line(data[ellipse_index * 24: (ellipse_index + span) * 24, 2], time[ellipse_index * 24: (ellipse_index + span) * 24], opts={'markersize': 4}, win='Axis'+str(axis))

In [122]:
sat_id = 1
plot_axis(sat_id, 0, 0, 11)
plot_axis(sat_id, 1, 0, 11)
plot_axis(sat_id, 2, 0, 11)

In [123]:
import torch
import torch.nn as nn

In [173]:
from torch.utils.data import DataLoader
import torch.optim as optim

In [357]:
class SatNet(nn.Module):

    def __init__(self, name,omega, mean, diff):
        super(SatNet, self).__init__()
        self.name = name
        self.A = nn.Parameter(diff)
        self.B = nn.Parameter(diff)
        self.W = nn.Parameter(omega)
        self.P1 = nn.Parameter(torch.randn(1))
        self.P2 = nn.Parameter(torch.randn(1))
        self.off = nn.Parameter(mean)


    def forward(self, t):

        x = self.A*torch.cos(self.W*t + self.P1) + self.B*torch.cos(2.0*self.W*t + self.P2) + self.off
        vx = -self.A*self.W*torch.sin(self.W*t + self.P1) - 2.0*self.B*self.W*torch.sin(self.W*t + self.P2) 

        return x, vx

In [345]:
t = torch.zeros((2))
model = SatNet("abc")
# print("B ",model.off.item())
# print("A ",model.A.item())
# print("w ",model.W.item())
# print("phi ",model.P.item())
print(model(t))


TypeError: __init__() missing 2 required positional arguments: 'mean' and 'diff'

In [346]:
zeroes = {}
for i in range(n_sat):
    times = train_data[train_data['sat_id']==i].loc[:,'epoch']
    zeroes[i] = np.min(times)
#zeroes

In [347]:
def smape(satellite_predicted_values, satellite_true_values): 
    # the division, addition and subtraction are pointwise 
    return np.mean(np.abs((satellite_predicted_values - satellite_true_values) 
        / (np.abs(satellite_predicted_values) + np.abs(satellite_true_values))))

In [386]:
div = 10000

def train(train_data):
    
    sat_net = {}
    
    for i in [150]:
        sat_data = train_data[train_data['sat_id']==i].loc[:, ['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim', 'epoch']].values
        sat_net[i] = {}
        
        
        data_ = DataLoader(sat_data, batch_size=24)
        data_iters = iter(data_)
        for j, dat in enumerate(data_iters):
            omega_ = 2*div*np.pi/(torch.max(dat[:, 6])-torch.min(dat[:, 6]))
            sat_net[i][j] = {'x': SatNet(name='x', omega=omega_, mean=torch.mean(dat[:,0]/div), diff=0.5/div*(torch.max(dat[:,0]) - torch.min(dat[:,0]))).float(), 
                             'y': SatNet(name='y', omega=omega_, mean=torch.mean(dat[:,1]/div), diff=0.5/div*(torch.max(dat[:,1]) - torch.min(dat[:,1]))).float(), 
                             'z': SatNet(name='z', omega=omega_, mean=torch.mean(dat[:,2]/div), diff=0.5/div*(torch.max(dat[:,2]) - torch.min(dat[:,2]))).float()}
            opt_x = optim.Adam(sat_net[i][j]['x'].parameters(), lr=0.05, betas = (0.9,0.999))
            opt_y = optim.Adam(sat_net[i][j]['y'].parameters(), lr=0.05, betas = (0.9,0.999))
            opt_z = optim.Adam(sat_net[i][j]['z'].parameters(), lr=0.05, betas = (0.9,0.999))
            
            last_loss = 0
            for its in range(10000):
                opt_x.zero_grad()
                opt_y.zero_grad()
                opt_z.zero_grad()
                
                time = dat[:,6]/div
                time = time.float()
                
                x_pred, vx_pred = sat_net[i][j]['x'](time)
                y_pred, vy_pred = sat_net[i][j]['y'](time)
                z_pred, vz_pred = sat_net[i][j]['z'](time)
                
                
                loss_x = torch.mean((x_pred - dat[:,0].float()/div)**2) 
                loss_y = torch.mean((y_pred - dat[:,1].float()/div)**2) 
                loss_z  = torch.mean((z_pred - dat[:,2].float()/div)**2)
                
                loss_vx = torch.mean((vx_pred - dat[:,3].float())**2) 
                loss_vy = torch.mean((vy_pred - dat[:,4].float())**2) 
                loss_vz =  torch.mean((vz_pred - dat[:,5].float())**2)
                
                loss_x_net = loss_x + loss_vx
                loss_y_net = loss_y + loss_vy
                loss_z_net = loss_z + loss_vz
                
                loss_x_net.backward()
                loss_y_net.backward()
                loss_z_net.backward()
                
                opt_x.step()
                opt_y.step()
                opt_z.step()
                
                
                
                if its % 1000 == 0:
                    print("Satellite Number {} Ellipse Number {} Loss x {} Loss y {} Loss z {}".format(i,j,loss_x.item(),\
                                                                                                loss_y.item(),loss_z.item()))
                   
                    print("Smape Loss X {} Smape Y {} Smape Z {}".format(smape(x_pred.detach().numpy()*div,dat[:,0].numpy()),
                                                                         smape(y_pred.detach().numpy()*div,dat[:,1].numpy()),
                                                                         smape(z_pred.detach().numpy()*div,dat[:,2].numpy())
                                                                            )
                         )
                
    return sat_net 

In [387]:
sat_nets = train(train_data)

Satellite Number 150 Ellipse Number 0 Loss x 36.12395477294922 Loss y 8.856234550476074 Loss z 6.997781276702881
Smape Loss X 0.7960624152518454 Smape Y 0.4372787480822833 Smape Z 0.5373274700909129
Satellite Number 150 Ellipse Number 0 Loss x 0.1360863298177719 Loss y 0.12732772529125214 Loss z 0.02119872160255909
Smape Loss X 0.058213176472154925 Smape Y 0.0699211355684911 Smape Z 0.0702646029831856
Satellite Number 150 Ellipse Number 0 Loss x 0.13608558475971222 Loss y 0.1273275911808014 Loss z 0.020213643088936806
Smape Loss X 0.0582129114532472 Smape Y 0.06992110062144043 Smape Z 0.06836079185922816
Satellite Number 150 Ellipse Number 0 Loss x 0.13608555495738983 Loss y 0.12732776999473572 Loss z 0.019750364124774933
Smape Loss X 0.05821287096545652 Smape Y 0.06992111407804251 Smape Z 0.06776253154112193
Satellite Number 150 Ellipse Number 0 Loss x 0.13608552515506744 Loss y 0.12784133851528168 Loss z 0.01972309686243534
Smape Loss X 0.05821302930431674 Smape Y 0.06995266690580486

KeyboardInterrupt: 

In [384]:
x_pred.shape

NameError: name 'x_pred' is not defined

In [274]:
sat_data = train_data[train_data['sat_id']==0].loc[:, ['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim', 'epoch']].values

In [137]:
x = sat_data[:,0]
y = sat_data[:,1]
z = sat_data[:,2]

In [138]:
vx = sat_data[:,3]
vy = sat_data[:,4]
vz = sat_data[:,5]

In [142]:
torch.from_numpy(sat_data)

tensor([[-8.8431e+03,  1.3138e+04, -2.0742e+04,  ..., -3.8049e+00,
         -2.0241e+00,  0.0000e+00],
        [-1.0556e+04,  1.6493e+03, -2.4473e+04,  ..., -4.2698e+00,
         -6.1647e-01,  2.8030e+03],
        [-1.0572e+04, -1.0146e+04, -2.4271e+04,  ..., -4.0468e+00,
          7.1877e-01,  5.6060e+03],
        ...,
        [ 1.0663e+04,  2.4548e+04,  2.3721e+04,  ...,  1.7154e+00,
         -2.6368e+00,  2.6713e+06],
        [ 7.1828e+03,  2.8396e+04,  1.5554e+04,  ...,  9.8470e-01,
         -3.1732e+00,  2.6741e+06],
        [ 3.0996e+03,  2.9798e+04,  6.0831e+03,  ..., -3.3515e-02,
         -3.5448e+00,  2.6769e+06]], dtype=torch.float64)

In [150]:
dataloader = DataLoader(sat_data, batch_size = 24)

In [151]:
dataset_iter = iter(dataloader)

In [154]:
k = next(dataset_iter).shape

In [155]:
k

torch.Size([24, 7])